# DCIR Setup Leveraging EDB Configuration Format

## Perform imports and define constants

Perform required imports.

In [1]:
import json
import os
import tempfile

from ansys.aedt.core import Hfss3dLayout
from ansys.aedt.core.examples.downloads import download_file
from pyedb import Edb

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NG_MODE = False

Download the example BGA Package design.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")
file_edb = download_file(source=r"edb/BGA_Package.aedb", local_path=temp_folder.name)

## Load example layout

In [4]:
edbapp = Edb(file_edb, edbversion=AEDT_VERSION)

C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\misc\decorators.py:55: UserWarning: Argument `edbversion` is deprecated for method `Edb`; use `version` instead.
  warnings.warn(
C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\generic\design_types.py:301: UserWarning: Your ANSYS AEDT version is eligible to gRPC version.You might consider switching to that version for better user experience.For more information please check this link: https://edb.docs.pyansys.com/version/dev/grpc_api/index.html
  warnings.warn(GRPC_GENERAL_WARNING, UserWarning)


PyEDB INFO: Star initializing Edb 10:25:59.682622


PyEDB INFO: Edb version 2025.2


PyEDB INFO: Logger is initialized. Log file is saved to C:\Users\ansys\AppData\Local\Temp\pyedb_ansys.log.


PyEDB INFO: legacy v0.67.1


...

PyEDB INFO: Database BGA_Package.aedb Opened in 2025.2


PyEDB INFO: Cell BGA_Package Opened


PyEDB INFO: Builder was initialized.


PyEDB INFO: open_edb completed in 8.6962 seconds.


PyEDB INFO: EDB initialization completed in 8.7602 seconds.


## Create config file

Define Component with solderballs.

In [5]:
cfg_components = [
    {
        "reference_designator": "FCHIP",
        "part_type": "other",
        "solder_ball_properties": {
            "shape": "cylinder",
            "diameter": "0.1mm",
            "height": "0.085mm",
            "chip_orientation": "chip_up",
        },
        "port_properties": {
            "reference_offset": 0,
            "reference_size_auto": False,
            "reference_size_x": 0,
            "reference_size_y": 0,
        },
    },
    {
        "reference_designator": "BGA",
        "part_type": "io",
        "solder_ball_properties": {
            "shape": "cylinder",
            "diameter": "0.45mm",
            "height": "0.45mm",
            "chip_orientation": "chip_down",
        },
        "port_properties": {
            "reference_offset": 0,
            "reference_size_auto": False,
            "reference_size_x": 0,
            "reference_size_y": 0,
        },
    },
]

Define Pin Groups on Components.

In [6]:
cfg_pin_groups = [
    {"name": "BGA_VSS", "reference_designator": "BGA", "net": "VSS"},
    {"name": "BGA_VDD", "reference_designator": "BGA", "net": "VDD"},
]

Define sources.

In [7]:
cfg_sources = [
    {
        "name": "FCHIP_Current",
        "reference_designator": "FCHIP",
        "type": "current",
        "magnitude": 2,
        "distributed": True,
        "positive_terminal": {"net": "VDD"},
        "negative_terminal": {"nearest_pin": {"reference_net": "VSS", "search_radius": 5e-3}},
    },
    {
        "name": "BGA_Voltage",
        "reference_designator": "BGA",
        "type": "voltage",
        "magnitude": 1,
        "positive_terminal": {"pin_group": "BGA_VDD"},
        "negative_terminal": {"pin_group": "BGA_VSS"},
    },
]

Define DCIR setup.

In [8]:
cfg_setups = [
    {
        "name": "siwave_dc",
        "type": "siwave_dc",
        "dc_slider_position": 1,
        "dc_ir_settings": {"export_dc_thermal_data": True},
    }
]

Create final configuration.

In [9]:
cfg = {
    "components": cfg_components,
    "sources": cfg_sources,
    "pin_groups": cfg_pin_groups,
    "setups": cfg_setups,
}

Create the config file.

In [10]:
file_json = os.path.join(temp_folder.name, "edb_configuration.json")
with open(file_json, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

## Apply Config file

Apply configuration to the example layout

In [11]:
edbapp.configuration.load(config_file=file_json)
edbapp.configuration.run()

PyEDB INFO: Updating nets finished. Time lapse 0:00:00


PyEDB INFO: Updating components finished. Time lapse 0:00:00.015726


PyEDB INFO: Creating pin groups finished. Time lapse 0:00:00.126913


PyEDB INFO: Placing sources finished. Time lapse 0:00:05.223978


PyEDB INFO: Applying materials finished. Time lapse 0:00:00


...

PyEDB INFO: Placing ports finished. Time lapse 0:00:00


PyEDB INFO: Placing terminals completed in 0.0000 seconds.


PyEDB INFO: Placing probes finished. Time lapse 0:00:00


PyEDB INFO: Applying operations completed in 0.0000 seconds.


True

Save and close EDB.

In [12]:
edbapp.save()
edbapp.close()

PyEDB INFO: Save Edb file completed in 0.0315 seconds.


PyEDB INFO: Close Edb file completed in 0.0161 seconds.


True

The configured EDB file is saved in a temp folder.

In [13]:
print(temp_folder.name)

C:\Users\ansys\AppData\Local\Temp\tmpy7hwvljr.ansys


## Load edb into HFSS 3D Layout.

In [14]:
h3d = Hfss3dLayout(edbapp.edbpath, version=AEDT_VERSION, non_graphical=NG_MODE, new_desktop=True)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_ff90a110-1113-46f9-b8b8-d653d5d3873e.log is enabled.


...

PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


PyAEDT INFO: EDB folder C:\Users\ansys\AppData\Local\Temp\tmpy7hwvljr.ansys\edb\BGA_Package.aedb has been imported to project BGA_Package


PyAEDT INFO: Active Design set to 0;BGA_Package


PyAEDT INFO: Active Design set to 0;BGA_Package


PyAEDT INFO: Aedt Objects correctly read


Solve.

In [15]:
h3d.analyze(setup="siwave_dc")

PyAEDT INFO: Project BGA_Package Saved correctly


PyAEDT INFO: Solving design setup siwave_dc


PyAEDT INFO: Design setup siwave_dc solved correctly in 0.0h 0.0m 30.0s


True

Plot insertion loss.

In [16]:
h3d.post.create_fieldplot_layers_nets(layers_nets=[["VDD_C1", "VDD"]], quantity="Voltage", setup="siwave_dc")

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpy7hwvljr.ansys\edb\BGA_Package.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmpy7hwvljr.ansys\edb\BGA_Package.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.0890190601348877


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


...

PyEDB INFO: EDB initialization completed in 0.0793 seconds.


PyAEDT INFO: EDB loaded.


PyAEDT INFO: Active Design set to 0;BGA_Package


C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\generic\design_types.py:301: UserWarning: Your ANSYS AEDT version is eligible to gRPC version.You might consider switching to that version for better user experience.For more information please check this link: https://edb.docs.pyansys.com/version/dev/grpc_api/index.html
  warnings.warn(GRPC_GENERAL_WARNING, UserWarning)


Class: ansys.aedt.core.visualization.post.field_data.FieldPlot

Shut Down Electronics Desktop

In [17]:
h3d.close_desktop()

PyAEDT INFO: Desktop has been released and closed.


True

All project files are saved in the folder ``temp_file.dir``. If you've run this example as a Jupyter notebook you
can retrieve those project files.